# TO-DO
1. assign the loadlfp output to mtf output 
1. assign dims to xarray++
1. then build a frame
1. read MTF doc as to how to apply mtf to post-analysis moving data (seems to be doable, forgot to set default params, eh)
2. Pipemodule: make a factory method, and change the params
3. nd a status tracker as we go along the th. right  


# Notes to clarify my fuzzy thinking

1. should frame take module as class and implement its own logic? 
    1. so, it can create classes inside itself.
    2. bad news is that it would increate computational complexity, eh. 
    
2. pipeframe: should the run_mod class handle the parameters. 

In [2]:
%config IPCompleter.greedy=True

import xarray as xr
from engines_interfaces import PipeModule, PipeFrame
from aopy import datareader
import numpy as np

print('import succeeded')

import succeeded


Q: can w automate t whole process o returning a fxn? 
t desgn pat is a factory pattern srt of th.
i.e a cls t builds mr cls, fun, wil lok into thz

# Build and test the load trial lfp fxn

##  cast loadlfp into a module

In [3]:
# f now, just inherit the PipModule
dim_types = ('TRIAL', 'ELECTRODE', 'FREQ', "TIME")

"""
# TO-DO
-test the thing
-cast this into a frame

"""

# thz is jst a wraper f


class PipeTrialLfpLoader(PipeModule):
    """

    """
    
    output_dims = ('TRIAL', 'ELECTRODE', 'TIME')

    _params_name_ordered = ('monkeyDrive', 'trials',)
    _params_name_KW = ('task_field', 'bn',
                        'microdrive_name', 'file_type',
                        'verbose')
        

    def __init__(self):
        super().__init__()

        """
        self.output_dims = ('TRIAL', 'ELECTRODE', 'TIME')

        self._params_name_ordered = ('monkeyDrive', 'trials',)
        self._params_name_KW = ('task_field', 'bn',
                                'microdrive_name', 'file_type',
                                'verbose')
        
        """
        self._params_val_ordered_dict =  dict()
        self._params_val_KW_dict = dict()


    def run_mod(self):
        # adding more logic
        # according to the flag, we don't h to run twice, right.
        if not self._data_available:
            _data = datareader.read_trials_lfp(self._params_val_ordered_dict['monkeyDrive'],
                                               self._params_val_ordered_dict['trials'],
                                               **self._params_val_KW_dict)

            # build xarray 1) cast data into xrray. 2) assign output dims into the dims
            self._output_xarray = xr.DataArray(_data,
                                         dims=self.output_dims)

            # assemble xarray data.
            self._data_available = True

        # TO-DO if it is okay py c chk self._params_val_dict is treated lk **kwarg
        # self._da_xarray = xr.DataArray(_data, dims = self.output_dims)

##  test the load frame fxn, eh



In [ ]:
# get a frame srt of th.   

trialLfp_loader = PipeTrialLfpLoader()

print(trialLfp_loader._params_name_ordered)
print(trialLfp_loader._params_name_KW )


monkeyDrive = r'E:\OneDrive - UW\projects\Brain EEG\data'
import json
rec_date = '180328'
mat_folder = 'mat'
file_name = 'Trials'
JSON_EXT = '.json'
trial_file_loc = monkeyDrive + '\\'\
                + rec_date +'\\'\
                + mat_folder + '\\'\
                + file_name + JSON_EXT

with open(trial_file_loc,'r') as f:
    trials = json.load(f)
#
trialLfp_args = {
    'monkeyDrive': monkeyDrive,
    'trials':trials,
    'task_field': 'ReachStart',
    'bn': np.array([-300, 500]),
    'microdrive_name': 'LM1_ECOG_3', 
    'file_type':'lfp',
    'verbose':False
}


print(trialLfp_args.keys())

## now mimick the actual fxn flow
1. check if ordered args exist in the args
2. assign the params to the internal parameter list


In [ ]:
#check compatibility of course
# this is a sort of flow
trialLfp_loader.check_ordered_args_exist(trialLfp_args)
# now we can run the srt of thing, eh?

# assign params value
trialLfp_loader.assign_params_value(trialLfp_args)

## call run module to run the module

In [ ]:
# TO-DO
# write a test to check how the program behave:
# arguments are not checked
# write a flow diagram srt o thing, eh.

trialLfp_loader.run_mod()

## explore the parameters

In [ ]:
# get the xarray
output_array = trialLfp_loader.get_data_xarray()
output_array.dims

this actually really well,
sd we explain to Amy what is happening, here?  blame interdependancies of our root cause,eh. 
this is very clear, we wil see if it act works or not,eh

## explore xarray params

In [ ]:
print(f'output_array.value type {type(output_array.data)}')

print(f'output_array.shape type {(output_array.shape)}')

sel_range = dict(TIME=slice(200))
output_array[sel_range]





# adapt MTF from nitime
use multitaper meth as an eg.
nitime: time series anal in python
1. used in BMI3D, in fact, this is wher I fd abot it. 
can find the exact def in:
https://github.com/nipy/nitime/blob/6f95beabf337a16ed882067f2ea3d0b32399c901/nitime/algorithms/spectral.py

## test out MTF function

In [ ]:

# for the input array.
# time is assumed to be on the last axis.
# which is clearly the case,
# it just converts to time.

In [ ]:

trial_xarray = output_array.isel(TRIAL=0, ELECTRODE=slice(2))

freqs, psd_est, var_or_nu = tsa.multi_taper_psd(trial_xarray.data,
                                                Fs=1000)  # magic!
# we h an array, because, we pass on an xarray and it does not have reshape.

In [ ]:
print(trial_xarray.data.shape)
print(trial_xarray.dims)

print(psd_est.shape)
print(freqs.shape)

## cast MTF into a  pipemodule

In [4]:
# I screwed big time in this,eht

# thz is jst a wraper f
import nitime.algorithms as tsa

class PipeMultiTaperEstimater(PipeModule):
    """

    """
    input_dims = ('TRIAL', 'ELECTRODE','TIME')
    output_dims = ('TRIAL', 'ELECTRODE','FREQ')
    
    _params_name_ordered = ('DATA',)
    _params_name_kwd = ('Fs', 'Nw')

    def __init__(self):
        self.mod_name = self.__class__.__name__

        
        """
                # ASSUME: this is the output dims for tsa.
        self.output_dims = ('TRIAL', 'ELECTRODE','FREQ')

        self._params_name_ordered = ('DATA',)
        self._params_name_kwd = ('Fs', 'Nw')
        
        """
        self._params_val_ordered_dict =  dict()
        self._params_val_KW_dict = dict()
        
        self._data_available = False 
    

    def run_mod(self):
        # for mor detail go to nitime.algorithms.spectral
        if not self._data_available:
            freqs, psd_est, var_or_nu = tsa.multi_taper_psd(self._params_val_dict['DATA'].values,
                                                            **self._params_val_KW_dict)
            # build xarray 1) cast data into xrray. 2) assign output dims into the dims
            self._output_xarray = xr.DataArray(psd_est,
                                         dims=self.output_dims)
            self._data_available = True


## test the mtf module

In [ ]:
mtf = PipeMultiTaperEstimater()

print(mtf._params_name_ordered)


trial_xarray = output_array.isel(TRIAL= slice(2), ELECTRODE=slice(2))

print(trial_xarray.shape)

#this is where the magic happens, right? 
mtf_args = {
    'DATA':trial_xarray,
    'Fs': 1000,
}


In [ ]:
#check compatibility of course
# this is a sort of flow
print(mtf.check_ordered_args_exist(mtf_args))
# now we can run the srt of thing, eh?

# assign params value
mtf.assign_params_value(mtf_args)

print(mtf._params_val_KW_dict)

## can actually run the params

In [ ]:
mtf.run_mod()
mtf_output = mtf.get_data_xarray()
print(type(mtf_output))

# link trialLfp to MTF using frames

assume they are welrtn A tsted, whch thy n

## build a loadlfp frame

In [5]:
trialLFPload_frame = PipeFrame(frame_type = 'input', module = PipeTrialLfpLoader)


## test out the loadlfp frame, eh

In [6]:
monkeyDrive = r'E:\OneDrive - UW\projects\Brain EEG\data'
import json
rec_date = '180328'
mat_folder = 'mat'
file_name = 'Trials'
JSON_EXT = '.json'
trial_file_loc = monkeyDrive + '\\'\
                + rec_date +'\\'\
                + mat_folder + '\\'\
                + file_name + JSON_EXT

with open(trial_file_loc,'r') as f:
    trials = json.load(f)
#
trialLfp_args = {
    'monkeyDrive': monkeyDrive,
    'trials':trials,
    'task_field': 'ReachStart',
    'bn': np.array([-300, 500]),
    'microdrive_name': 'LM1_ECOG_3', 
    'file_type':'lfp',
    'verbose':False
}


print(trialLfp_args.keys())

# go through the process, eh
trialLFPload_frame._create_instances()
trialLFPload_frame._assign_params_to_mod(trialLfp_args)
trialLFPload_frame.run_frame()

frame_output_dict = trialLFPload_frame.get_frame_data()
#we can get data, we do not have to, at this time. 
frame_output_list = trialLFPload_frame.get_frame_data()
frame_output_list

dict_keys(['monkeyDrive', 'trials', 'task_field', 'bn', 'microdrive_name', 'file_type', 'verbose'])


[<bound method PipeModule.get_data_xarray of <__main__.PipeTrialLfpLoader object at 0x0000022E5D5EC1C8>>]

## set up a mtf frame

In [ ]:
# ASSUM PipeTrialLfpLoader A PipeMultiTaperEstimater h b blt to t spec
# ASSUM: PipeFrame h b blt, unver of crse

# now, bld two frames
# build t estimater frame 1st


mtf_estimaters_frame = PipeFrame(frame_type = 'proc', module = PipeMultiTaperEstimater)

# INS,
# d t same th. for ld
# aft tht, run check compat


##  test out the frame

In [ ]:
mtf_parameters = {
    'Fs': 1000,
    'NW': np.arange(1, 10),  # normalized  half-bandwidth
    'BW': np.arange(10, 20)  # sampl rela bandwidth
}

mtf_estimaters_frame._create_instances()
mtf_estimaters_frame._assign_params_to_mod(mtf_parameters)
mtf_estimaters_frame.run_frame()

# set up a pipeline

# some debugging tries
they don't add to the story at this time

In [ ]:
#some one provided a function to do this. 

def reorder_dims(darray, dim1, dim2):
    """
    Interchange two dimensions of a DataArray in a similar way as numpy's swap_axes
    """
    dims = list(darray.dims)
    assert set([dim1,dim2]).issubset(dims), 'dim1 and dim2 must be existing dimensions in darray'
    ind1, ind2 = dims.index(dim1), dims.index(dim2)
    dims[ind2], dims[ind1] = dims[ind1], dims[ind2]
    return darray.transpose(*dims)


#now we can do transpose
new_dims_in_tuple = ('TIME', 'ELECTRODE', 'TRIAL')
print(new_dims_in_tuple)

print(f'shape before transpose:{output_array.dims}')

output_array = output_array.transpose(*new_dims_in_tuple)

print(f'shape after transpose:{output_array.dims}')


In [ ]:
dims_as_list = list(output_array.dims)

print('list of dim names before remove time:')
print(dims_as_list)

dims_as_list.remove('TIME')
print('list after remove time:')
print(dims_as_list)

dims_as_list.append('TIME')
print('list after time to the last pos:')
print(dims_as_list)

new_dims_in_tuple =  tuple(dims_as_list)

new_dims_in_tuple

## check if an object is a class

In [ ]:
class x:
    pass

class y(x):
    pass
#y = 1

temp_list = [x, y]

type_check_list = [isinstance(i, type) for i in temp_list]
type_check_list



In [ ]:
# check if insubclass works
print(f'check all lists are classes or types: {type_check_list}')

x_ins = x()
issubclass(x, x)